In [9]:
# run if running locally
path = ''

In [10]:
#! remember to remove
%load_ext autoreload
%autoreload 2

import pandas as pd

from funcs.preparation import Preprocessing, FeatureSelection
from funcs.clustering import Clustering
from funcs.integration_exploration import Integration, Exploration

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#setting seaborn visual style in plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
pd.set_option('display.max_rows', 1000)

In [12]:
metricFeatures: list[str] = ['Application order','Previous qualification score','Entry score','Age at enrollment','N units credited 1st period',
       'N units taken 1st period', 'N scored units 1st period',
       'N units approved 1st period', 'Average grade 1st period',
       'N unscored units 1st period', 'N units credited 2nd period',
       'N units taken 2nd period', 'N scored units 2nd period',
       'N units approved 2nd period', 'Average grade 2nd period',
       'N unscored units 2nd period','Social Popularity']
categoricalFeatures: list[str] =  ['Application mode','Marital status','Course','Previous qualification','Nationality','Mother\'s qualification',
       'Father\'s qualification','Mother\'s occupation','Father\'s occupation']
boolFeatures: list[str] = ['Morning shift participation','Displaced','Special needs','Debtor','Regularized Fees','Gender_Male','External Funding','International']

In [13]:
academicFeatures: list[str] = [
    'Entry score',
    'N units credited 1st period',
    'N units taken 1st period',
    'N scored units 1st period',
    'N units approved 1st period',
    'Average grade 1st period',
    'N unscored units 1st period',
    'N units credited 2nd period',
    'N units taken 2nd period',
    'N scored units 2nd period',
    'N units approved 2nd period',
    'Average grade 2nd period',
    'N unscored units 2nd period',
    'Success'
]

demographicFeatures: list[str] = [
    'Age at enrollment',
    'Marital status',
    'Course',
    'Previous qualification',
    'Nationality',
    'Mother\'s qualification',
    'Father\'s qualification',
    'Mother\'s occupation',
    'Father\'s occupation',
    'Special needs',
    'International',
    'Debtor',
    'Regularized Fees',
    'Gender_Male'
]

In [14]:
def runAll(
    metricFeatures: list[str],
    categoricalFeatures: list[str],
    boolFeatures: list[str],
    academicFeatures: list[str],
    demographicFeatures: list[str],
    removedAcademicFeatures: list[str] | list = [],
    removedDemographicFeatures: list[str] | list = [],
    path: str = '',
    *,
    describeDF: bool = False,
    featureSelection: bool = False,
    getPairPlot: bool = False,
    elbowGraph: bool = True, 
    silhouetteGraph: bool = True, 
    dendrogram: bool = True,
    clusterProfiling: bool = False,
    nClustersAcademic: int = 0,
    nClustersDemographic: int = 0,
) -> None:
    """_summary_

    Args:
        metricFeatures (list[str]): _description_
        categoricalFeatures (list[str]): _description_
        boolFeatures (list[str]): _description_
        academicFeatures (list[str]): _description_
        demographicFeatures (list[str]): _description_
        path (str, optional): _description_. Defaults to ''.
        describeDF (bool, optional): _description_. Defaults to False.
        getPairPlot (bool, optional): No, just don't please. Defaults to False.
        nClustersAcademic (int, optional): _description_. Defaults to 4.
        nClustersDemographic (int, optional): _description_. Defaults to 4.
    """    
    dataDF: pd.DataFrame = Integration.importdata(path, False)

    if describeDF:
        Exploration.describeData(dataDF, metricFeatures, categoricalFeatures, getBoxPlots=True)
        return

    dataDF, dataAcademic, dataDemographic, scaler = Preprocessing.runPreprocessing(dataDF, metricFeatures, boolFeatures, academicFeatures, demographicFeatures, removedAcademicFeatures, removedDemographicFeatures, grouping="high")

    if featureSelection:
        if getPairPlot:
            FeatureSelection.pairPlots(dataAcademic, "Academic")
            FeatureSelection.pairPlots(dataDemographic, "Demographic")

        FeatureSelection.checkCorr(dataAcademic)
        FeatureSelection.checkCorr(dataDemographic)
        return

    if nClustersAcademic == 0:
        Clustering.somWrapper(dataAcademic, "Academic", nClustersAcademic)
        return
    
    dataAcademic = Clustering.somWrapper(dataAcademic, "Academic", nClustersAcademic)

    dataDemographic = Clustering.runKMeans(dataDemographic,nClustersDemographic, elbowGraph=elbowGraph, silhouetteGraph=silhouetteGraph, dendrogram=dendrogram)

    if clusterProfiling:
        FeatureSelection.clusterProfiles(dataAcademic.drop("bmu", axis=1), ["label"], (32, 10))
        FeatureSelection.getVariableClusterGraphs(dataAcademic)
        FeatureSelection.clusterProfiles(dataDemographic, ["label"], (32, 10))
        FeatureSelection.getVariableClusterGraphs(dataDemographic)
        return

    display(dataAcademic.groupby(['label']).describe().T)
    display(dataDemographic.groupby(['label']).describe().T)
    display(Clustering.mergePerspectives(dataDF, dataAcademic, dataDemographic, scaler))
    

In [15]:
removedAcademicFeatures: list[str] = [
    'Average units taken',
    'Average units credited',
    'Average unscored units'
]

removedDemographicFeatures: list[str] = [
    'Special needs',
    'International',
    'Debtor',
    'Regularized Fees',
    'Marital status-together',
    'Marital status-nan',
    'Father\'s qualification-None',
    'Mother\'s occupation-Service Industry',
    'Mother\'s occupation-White collar Jobs',
    'Father\'s occupation-Professional Fields',
    'Father\'s occupation-Service Industry',
    'Father\'s occupation-Technical and Skilled Trades',
    'Father\'s occupation-White collar Jobs',
    'Father\'s occupation-nan',
    'Mother\'s qualification-None',
    'Mother\'s qualification-nan',
    'Mother\'s occupation-Technical and Skilled Trades',
    'Mother\'s occupation-nan',
    'Father\'s occupation-Security and Armed Forces',
    'Father\'s qualification-nan',
    'Nationality-Angolan',
    'Nationality-Brazilian',
    'Nationality-Cape Verdean',
    'Nationality-Colombian',
    'Nationality-Cuban',
    'Nationality-Dutch',
    'Nationality-English',
    'Nationality-German',
    'Nationality-Guinean',
    'Nationality-Italian',
    'Nationality-Lithuanian',
    'Nationality-Mexican',
    'Nationality-Moldova',
    'Nationality-Mozambican',
    'Nationality-Portuguese',
    'Nationality-Romanian',
    'Nationality-Russian',
    'Nationality-Santomean',
    'Nationality-Spanish',
    'Nationality-Turkish',
    'Nationality-Ukrainian',
    'Nationality-nan',
    'Course-Advertising and Marketing Management',
    'Course-Agronomy',
    'Course-Animation and Multimedia Design',
    'Course-Basic Education',
    'Course-Biofuel Production Technologies',
    'Course-Communication Design',
    'Course-Equinculture',
    'Course-Informatics Engineering',
    'Course-Journalism and Communication',
    'Course-Management',
    'Course-Management (evening attendance)',
    'Course-Nursing',
    'Course-Oral Hygiene',
    'Course-Social Service',
    'Course-Social Service (evening attendance)',
    'Course-Tourism',
    'Course-Veterinary Nursing',
    'Course-nan',
    'Previous qualification-Higher',
    'Previous qualification-Lower',
    'Previous qualification-Middle',
    'Previous qualification-nan'
]

In [ ]:
dataDF: pd.DataFrame = Integration.importdata(path, False)
dataDF, dataAcademic, dataDemographic, scaler = Preprocessing.runPreprocessing(dataDF, metricFeatures, boolFeatures, academicFeatures, demographicFeatures, removedAcademicFeatures, removedDemographicFeatures, grouping="high")
dataDemographic = Clustering.runKMeans(dataDemographic,nClustersDemographic, elbowGraph=elbowGraph, silhouetteGraph=silhouetteGraph, dendrogram=dendrogram)

In [16]:
runAll(
    metricFeatures,
    categoricalFeatures,
    boolFeatures,
    academicFeatures,
    demographicFeatures,
    removedAcademicFeatures,
    removedDemographicFeatures,
    path,
    describeDF=False,
    getPairPlot=False,
    elbowGraph=False,
    silhouetteGraph=False,
    dendrogram=False,
    clusterProfiling=False,
    nClustersAcademic=3,
    nClustersDemographic=3,
)

label                                   0           1            2
Entry score            count  2134.000000  784.000000  1327.000000
                       mean      0.353097    0.321340     0.318337
                       std       0.149124    0.174220     0.139462
                       min       0.000000    0.000000     0.000000
                       25%       0.256842    0.210526     0.230000
                       50%       0.339474    0.303158     0.305263
                       75%       0.433684    0.411842     0.398947
                       max       1.000000    0.931579     1.000000
Success                count  2134.000000  784.000000  1327.000000
                       mean      0.987816    0.084821     0.247174
                       std       0.089755    0.238689     0.250078
                       min       0.000000    0.000000     0.000000
                       25%       1.000000    0.000000     0.000000
                       50%       1.000000    0.000000     0.000000
                       75%       1.000000    0.000000     0.500000
                       max       1.000000    1.000000     0.500000
Average grades         count  2134.000000  784.000000  1327.000000
                       mean      0.687290    0.058833     0.632117
                       std       0.114766    0.122102     0.095635
                       min       0.000000    0.000000     0.264901
                       25%       0.653986    0.000000     0.600573
                       50%       0.694371    0.000000     0.637120
                       75%       0.743152    0.000000     0.681586
                       max       0.976331    0.430769     0.916853
Average scored units   count  2134.000000  784.000000  1327.000000
                       mean      0.457092    0.235395     0.543770
                       std       0.141491    0.222021     0.138671
                       min       0.000000    0.000000     0.000000
                       25%       0.375000    0.000000     0.450000
                       50%       0.450000    0.291667     0.533333
                       75%       0.508333    0.408333     0.641667
                       max       1.000000    0.883333     0.975000
Average units approved count  2134.000000  784.000000  1327.000000
                       mean      0.386812    0.017847     0.259321
                       std       0.112810    0.040931     0.106401
                       min       0.000000    0.000000     0.027778
                       25%       0.317460    0.000000     0.190476
                       50%       0.380952    0.000000     0.253968
                       75%       0.444444    0.000000     0.317460
                       max       0.964286    0.178571     0.817460

label                                                          0            1  \
Age at enrollment                             count  1380.000000  1666.000000   
                                              mean      0.152899     0.078881   
                                              std       0.166777     0.092138   
                                              min       0.018868     0.000000   
                                              25%       0.037736     0.018868   
                                              50%       0.075472     0.037736   
                                              75%       0.226415     0.075472   
                                              max       0.811321     0.849057   
Gender_Male                                   count  1380.000000  1666.000000   
                                              mean      0.335507     0.399760   
                                              std       0.472339     0.489996   
                                              min       0.000000     0.000000   
                                              25%       0.000000     0.000000   
                                              50%       0.000000     0.000000   
                                              75%       1.000000     1.000000   
                                              max       1.000000     1.000000   
Marital status-alone                          count  1380.000000  1666.000000   
                                              mean      0.850000     0.968187   
                                              std       0.357201     0.175554   
                                              min       0.000000     0.000000   
                                              25%       1.000000     1.000000   
                                              50%       1.000000     1.000000   
                                              75%       1.000000     1.000000   
                                              max       1.000000     1.000000   
Mother's qualification-Higher                 count  1380.000000  1666.000000   
                                              mean      0.003623     0.349340   
                                              std       0.060106     0.476905   
                                              min       0.000000     0.000000   
                                              25%       0.000000     0.000000   
                                              50%       0.000000     0.000000   
                                              75%       0.000000     1.000000   
                                              max       1.000000     1.000000   
Mother's qualification-Lower                  count  1380.000000  1666.000000   
                                              mean      0.882609     0.016807   
                                              std       0.322003     0.128585   
                                              min       0.000000     0.000000   
                                              25%       1.000000     0.000000   
                                              50%       1.000000     0.000000   
                                              75%       1.000000     0.000000   
                                              max       1.000000     1.000000   
Mother's qualification-Middle                 count  1380.000000  1666.000000   
                                              mean      0.044203     0.620048   
                                              std       0.205620     0.485520   
                                              min       0.000000     0.000000   
                                              25%       0.000000     0.000000   
                                              50%       0.000000     1.000000   
                                              75%       0.000000     1.000000   
                                              max       1.000000     1.000000   
Father's qualification-Higher  

final_groups                                                  0           1  \
Entry score                                   count  821.000000  546.000000   
                                              mean   129.308404  125.801099   
                                              std     14.725160   13.276038   
                                              min     98.100000   96.000000   
                                              25%    119.800000  117.225000   
                                              50%    128.000000  124.300000   
                                              75%    137.500000  132.900000   
                                              max    190.000000  190.000000   
Gender_Male                                   count  821.000000  546.000000   
                                              mean     0.285018    0.476190   
                                              std      0.451698    0.499891   
                                              min      0.000000    0.000000   
                                              25%      0.000000    0.000000   
                                              50%      0.000000    0.000000   
                                              75%      1.000000    1.000000   
                                              max      1.000000    1.000000   
Age at enrollment                             count  821.000000  546.000000   
                                              mean    20.325213   21.500000   
                                              std      4.225672    4.885571   
                                              min     17.000000   17.000000   
                                              25%     18.000000   19.000000   
                                              50%     19.000000   20.000000   
                                              75%     20.000000   22.000000   
                                              max     62.000000   53.000000   
Success                                       count  821.000000  546.000000   
                                              mean     1.981730    0.527473   
                                              std      0.151118    0.499703   
                                              min      0.000000    0.000000   
                                              25%      2.000000    0.000000   
                                              50%      2.000000    1.000000   
                                              75%      2.000000    1.000000   
                                              max      2.000000    1.000000   
Marital status-alone                          count  821.000000  546.000000   
                                              mean     0.969549    0.972527   
                                              std      0.171929    0.163606   
                                              min      0.000000    0.000000   
                                              25%      1.000000    1.000000   
                                              50%      1.000000    1.000000   
                                              75%      1.000000    1.000000   
                                              max      1.000000    1.000000   
Mother's qualification-Higher                 count  821.000000  546.000000   
                                              mean     0.328867    0.371795   
                                              std      0.470088    0.483727   
                                              min      0.000000    0.000000   
                                              25%      0.000000    0.000000   
                                              50%      0.000000    0.000000   
                                              75%      1.000000    1.000000   
                                              max      1.000000    1.000000   
Mother's qualification-Lower                  count  821.000000  546.000000   
                                              mean